In [74]:
#pseudo-random
import random

class KeyStream():
    def __init__(self, key=1):
        self.next = key
    
    def rand(self):
        self.next = (1103515245*self.next + 12345)% 2**31
        return self.next
    
    def get_key_bite(self):
        return self.rand() % 255
    
def encrypt(key, message):
    return bytes([message[i]^key for i in range(len(message))])
    
def transmit(cipher, likely):
    b = []
    for c in cipher:
        if random.randrange(0, likely) < 0.5:
            c = c^ 2**random.randrange(0,8)
        b.append(c)
    return bytes(b)
    
def modification(cipher):
    mod = [0]*len(cipher)
    mod[10] = ord(' ') ^ ord('1')
    mod[11] = ord(' ') ^ ord('0')
    mod[12] = ord('1') ^ ord('0')
    return bytes([mod[i]^cipher[i] for i in range(len(cipher))])

def get_key(message, cipher):
    return bytes([message[i]^cipher[i] for i in range(len(message))])

def crack(key_stream, cipher):
    lenght = min(len(key_stream), len(cipher))
    return bytes([key_stream[i]^cipher[i] for i in range(lenght)])
    

In [75]:
key = KeyStream(10)
message =  "Send Bob:   10$".encode()
print(message)
cipher = encrypt(key.get_key_bite(), message)
print(cipher)

b'Send Bob:   10$'
b'\xdf\xe9\xe2\xe8\xac\xce\xe3\xee\xb6\xac\xac\xac\xbd\xbc\xa8'


In [76]:
cipher_b = modification(cipher)
key = KeyStream(10)
message = encrypt(key.get_key_bite(), cipher_b)
print(message)

b'Send Bob: 1000$'


In [77]:
#this is eve
key2 = KeyStream(10)
message2 =  "This is eve secret".encode()
print(message2)
cipher2 = encrypt(key2.get_key_bite(), message2)
print(cipher2)

b'This is eve secret'
b'\xd8\xe4\xe5\xff\xac\xe5\xff\xac\xe9\xfa\xe9\xac\xff\xe9\xef\xfe\xe9\xf8'


In [78]:
#alice again
key2 = KeyStream(10)
message3 = "Hi BoB lets, meet and plan our stuff".encode()
cipher3 = encrypt(key2.get_key_bite(), message3)
print(cipher3)

b'\xc4\xe5\xac\xce\xe3\xce\xac\xe0\xe9\xf8\xff\xa0\xac\xe1\xe9\xe9\xf8\xac\xed\xe2\xe8\xac\xfc\xe0\xed\xe2\xac\xe3\xf9\xfe\xac\xff\xf8\xf9\xea\xea'


In [79]:
#eve decryption
#this is eve, she knows the message and seeks the key
eves_key_stream = get_key(message3, cipher3)
crack(eves_key_stream, cipher3)

b'Hi BoB lets, meet and plan our stuff'